<a href="https://colab.research.google.com/github/shahrzadJavadiKoushesh/Process-Outcome-Prediction-BachelorProject/blob/main/ProcessMiningBPIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Git Repository

In [ ]:
!wget -q https://raw.githubusercontent.com/tsunrise/colab-github/main/colab_github.py
import colab_github
colab_github.github_auth(persistent_key=True)


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Please go to https://github.com/settings/ssh/new to upload the following key: 
ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIGw64lNtNjD+dJXYjQNiF27iEuGzxAXbxhEszStsYjM3 root@1575d9f6bc52

Please use SSH method to clone repo.


In [7]:
!git clone git@github.com:shahrzadJavadiKoushesh/Process-Outcome-Prediction-BachelorProject.git



Cloning into 'Process-Outcome-Prediction-BachelorProject'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


# Reading Data



In [ ]:
from google.colab import drive

drive.mount('drive')

Mounted at drive


In [ ]:
# Use ijson to read data step by step and not all at once
!pip install ijson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import ijson
path = 'drive/MyDrive/ProcessMiningBachelorProject/BPIC19/BPIC19.jsonocel'
data = []

# Use ijson to read the file incrementally
with open(path, 'r') as f:
    for item in ijson.items(f, 'ocel:events'):
        data.append(item)

print(len(data))

1


In [ ]:
events_df = pd.DataFrame(data)
events_df

,1900003,1900004,1900005,1900006,1900007,1900008,1900009,1900010,1900011,1900012,...,321904,321905,321906,321907,321908,321909,321910,321911,321912,321913
0,"{'ocel:timestamp': '1948-01-26T22:59:00', 'oce...","{'ocel:timestamp': '1948-01-26T22:59:00', 'oce...","{'ocel:timestamp': '1948-01-26T22:59:00', 'oce...","{'ocel:timestamp': '1948-01-26T22:59:00', 'oce...","{'ocel:timestamp': '1948-01-26T22:59:00', 'oce...","{'ocel:timestamp': '1948-01-26T22:59:00', 'oce...","{'ocel:timestamp': '1948-01-26T22:59:00', 'oce...","{'ocel:timestamp': '1948-01-26T22:59:00', 'oce...","{'ocel:timestamp': '1948-01-26T22:59:00', 'oce...","{'ocel:timestamp': '1948-01-26T22:59:00', 'oce...",...,"{'ocel:timestamp': '2019-01-18T13:34:00', 'oce...","{'ocel:timestamp': '2019-02-01T22:59:00', 'oce...","{'ocel:timestamp': '2019-02-01T22:59:00', 'oce...","{'ocel:timestamp': '2019-04-30T22:59:00', 'oce...","{'ocel:timestamp': '2019-04-30T22:59:00', 'oce...","{'ocel:timestamp': '2019-10-03T22:59:00', 'oce...","{'ocel:timestamp': '2019-12-05T22:59:00', 'oce...","{'ocel:timestamp': '2019-12-05T22:59:00', 'oce...","{'ocel:timestamp': '2020-04-06T22:59:00', 'oce...","{'ocel:timestamp': '2020-04-09T22:59:00', 'oce..."


In [ ]:
print(events_df.head())

                                             1900003  \
0  {'ocel:timestamp': '1948-01-26T22:59:00', 'oce...   

                                             1900004  \
0  {'ocel:timestamp': '1948-01-26T22:59:00', 'oce...   

                                             1900005  \
0  {'ocel:timestamp': '1948-01-26T22:59:00', 'oce...   

                                             1900006  \
0  {'ocel:timestamp': '1948-01-26T22:59:00', 'oce...   

                                             1900007  \
0  {'ocel:timestamp': '1948-01-26T22:59:00', 'oce...   

                                             1900008  \
0  {'ocel:timestamp': '1948-01-26T22:59:00', 'oce...   

                                             1900009  \
0  {'ocel:timestamp': '1948-01-26T22:59:00', 'oce...   

                                             1900010  \
0  {'ocel:timestamp': '1948-01-26T22:59:00', 'oce...   

                                             1900011  \
0  {'ocel:timestamp': '1948-01-26T22:59:

In [ ]:
# Transpose the DataFrame to switch rows and columns
transposed_df = events_df.T
print(transposed_df.head())

                                                         0
1900003  {'ocel:timestamp': '1948-01-26T22:59:00', 'oce...
1900004  {'ocel:timestamp': '1948-01-26T22:59:00', 'oce...
1900005  {'ocel:timestamp': '1948-01-26T22:59:00', 'oce...
1900006  {'ocel:timestamp': '1948-01-26T22:59:00', 'oce...
1900007  {'ocel:timestamp': '1948-01-26T22:59:00', 'oce...


In [ ]:
flattened_data = []

for event_id, event_details in transposed_df.iterrows():
    event_details = event_details[0]

    # Extracting values
    timestamp = event_details['ocel:timestamp']
    activity = event_details['ocel:activity']

    # Extract nested vmap attributes
    vmap = event_details['ocel:vmap']

    # Combine all data into a single dictionary
    flattened_event = {
        'event_id': event_id,
        'timestamp': timestamp,
        'activity': activity,
        **vmap  # Unpack vmap attributes into the dictionary
    }

    flattened_data.append(flattened_event)

# Create a flattened DataFrame from the processed data
events_df = pd.DataFrame(flattened_data)

print(events_df.head())

  event_id            timestamp                   activity              ID  \
0  1900003  1948-01-26T22:59:00     Vendor creates invoice  65781719105536   
1  1900004  1948-01-26T22:59:00  Vendor creates debit memo  65777424138241   
2  1900005  1948-01-26T22:59:00     Vendor creates invoice  65777424138240   
3  1900006  1948-01-26T22:59:00     Vendor creates invoice  65794604007424   
4  1900007  1948-01-26T22:59:00  Vendor creates debit memo  65794604007425   

         cCompany     cDocType   cGR cGRbasedInvVerif               cID cItem  \
0  companyID_0000  Standard PO  True            False  4507004931_00020  20.0   
1  companyID_0000  Standard PO  True            False  4507004931_00010  10.0   
2  companyID_0000  Standard PO  True            False  4507004931_00010  10.0   
3  companyID_0000  Standard PO  True            False  4507004931_00050  50.0   
4  companyID_0000  Standard PO  True            False  4507004931_00050  50.0   

   ...         cPOID      cPurDocCat cSpendA

In [ ]:
file_path = 'drive/MyDrive/ProcessMiningBachelorProject/BPIC19/events_df.csv'
# Save the DataFrame to a CSV file
events_df.to_csv(file_path, index=False)

print(f'DataFrame saved to {file_path}')

# Pre-Processing

In [ ]:
import pandas as pd
dataset_address = 'drive/MyDrive/ProcessMiningBachelorProject/BPIC19/events_df.csv'
df = pd.read_csv(dataset_address)
df.head()

,event_id,timestamp,activity,ID,cCompany,cDocType,cGR,cGRbasedInvVerif,cID,cItem,...,cPOID,cPurDocCat,cSpendAreaText,cSpendClassText,cSubSPendAreaText,cVendor,cVendorName,eCumNetWorth,idx,resource
0,1900003,1948-01-26T22:59:00,Vendor creates invoice,65781719105536,companyID_0000,Standard PO,True,False,4507004931_00020,20.0,...,4.507005e+09,Purchase order,Sales,NPR,Products for Resale,vendorID_0670,vendor_0645,68.0,0.0,NONE
1,1900004,1948-01-26T22:59:00,Vendor creates debit memo,65777424138241,companyID_0000,Standard PO,True,False,4507004931_00010,10.0,...,4.507005e+09,Purchase order,Sales,NPR,Products for Resale,vendorID_0670,vendor_0645,325.0,1.0,NONE
2,1900005,1948-01-26T22:59:00,Vendor creates invoice,65777424138240,companyID_0000,Standard PO,True,False,4507004931_00010,10.0,...,4.507005e+09,Purchase order,Sales,NPR,Products for Resale,vendorID_0670,vendor_0645,325.0,2.0,NONE
3,1900006,1948-01-26T22:59:00,Vendor creates invoice,65794604007424,companyID_0000,Standard PO,True,False,4507004931_00050,50.0,...,4.507005e+09,Purchase order,Sales,NPR,Products for Resale,vendorID_0670,vendor_0645,102.0,3.0,NONE
4,1900007,1948-01-26T22:59:00,Vendor creates debit memo,65794604007425,companyID_0000,Standard PO,True,False,4507004931_00050,50.0,...,4.507005e+09,Purchase order,Sales,NPR,Products for Resale,vendorID_0670,vendor_0645,102.0,4.0,NONE


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['event_id'].plot(kind='hist', bins=20, title='event_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['ID'].plot(kind='hist', bins=20, title='ID')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['cItem'].plot(kind='hist', bins=20, title='cItem')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['cID'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='index', y='cID', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_21['activity'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_21, x='event_id', y='activity', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_22['cID'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_22, x='event_id', y='cID', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)